In [21]:
partial_prod_num = []
for i in range(1, 25):
    partial_prod_num.append(i)
    
for i in range(23, -1, -1):
    partial_prod_num.append(i)

print(partial_prod_num)
print()

# Follows 19, 13, 9, 6, 4, 3, 2
# Generate gates
d_seq = [19, 13, 9, 6, 4, 3, 2]
L_list = []
Lbit_list = partial_prod_num
for d in d_seq:
    curr_L = []
    carry = 0
    print('d : ', d)
    # Gate adding process
    for idx, i in enumerate(Lbit_list):
        # print('i : {}'.format(i))
        curr_tmp = []
        res = i
        i += carry
        carry = 0
        while i > d:
            if i - d > 1:
                curr_tmp += ['FA']
                i -= 2
                res -= 3
                carry += 1
            else:
                curr_tmp += ['HA']
                i -= 1
                res -= 2
                carry += 1
        curr_tmp += ['bp'] * res
        i = 0
        print('{0} : {1}'.format(idx, curr_tmp))
        curr_L.append(curr_tmp)
    
    #Gate term generating process
    terms = 0
    Lbit_tmp = []
    for idx, gates in enumerate(curr_L):
        next_terms = 0
        for gate in gates:     #FA or HA or bp
            if gate=='bp':
                terms += 1
            else:
                next_terms += 1
                terms += 1
        Lbit_tmp.append(terms)
        terms = next_terms
        
    Lbit_list = Lbit_tmp
    print('Terms : ', Lbit_list)
    L_list.append(curr_L)
    print()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

d :  19
0 : ['bp']
1 : ['bp', 'bp']
2 : ['bp', 'bp', 'bp']
3 : ['bp', 'bp', 'bp', 'bp']
4 : ['bp', 'bp', 'bp', 'bp', 'bp']
5 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp']
6 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
7 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
8 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
9 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
10 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
11 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
12 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
13 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
14 : ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp']
15 : ['bp', 'bp',

In [24]:
#Generate code! : behavior version
total_buffer = ''
stage_list = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7']
prev_stage_list = ['partial_products', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
stage_wire_list = L_list

#bit_idx : row, bit_num : col
#ha follows ha(sum, cout, a, b)
#fa follows fa(sum, cout, a, b, cin)

for stage, prev_stage, stage_wire in zip(stage_list, prev_stage_list, stage_wire_list):
    buffer = ''
    next_bit_num = 0
    inst_idx = 0
    print(stage, prev_stage)
    for bit_idx, gates in enumerate(stage_wire):
        bit_num = next_bit_num
        prev_bit_num = 0
        next_bit_num = 0
        prev_bit_idx = bit_idx
        for gate in gates:
            if gate=='bp':
                buffer += 'assign {0}[{1}][{2}] = {3}[{4}][{5}];\n'.format(stage, bit_num, bit_idx, prev_stage, prev_bit_num, prev_bit_idx)
                bit_num += 1
                prev_bit_num += 1
            elif gate=='HA':
                #HA gives carry to next bit_idx's bit
                buffer += 'assign {{ {0}[{7}][{2}+1], {0}[{1}][{2}] }} = {3}[{4}][{5}] + {3}[{4}+1][{5}];\n'.format(stage, bit_num, bit_idx, prev_stage, prev_bit_num, prev_bit_idx, inst_idx, next_bit_num)
                bit_num += 1
                next_bit_num += 1
                prev_bit_num += 2
                inst_idx += 1
            else:     #or FA
                #FA gives carry to next bit_idx's bit
                buffer += 'assign {{ {0}[{7}][{2}+1], {0}[{1}][{2}] }} = {3}[{4}][{5}] + {3}[{4}+1][{5}] + {3}[{4}+2][{5}];\n'.format(stage, bit_num, bit_idx, prev_stage, prev_bit_num, prev_bit_idx, inst_idx, next_bit_num)
                bit_num += 1
                next_bit_num += 1
                prev_bit_num += 3
                inst_idx += 1
        buffer += '\n'
    # print(buffer)
    total_buffer += buffer

L1 partial_products
L2 L1
L3 L2
L4 L3
L5 L4
L6 L5
L7 L6


In [25]:
f = open("dadda_tree_structure.txt", 'w')
f.write(total_buffer)
f.close()
